In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 4.0 MB 32.8 MB/s 
     |████████████████████████████████| 1.2 MB 60.6 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 880 kB 59.0 MB/s 
     |████████████████████████████████| 596 kB 52.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=03408a0a6c6081be0f64437255c88e40f419e5444b89d5d6b13361c521da8797
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7772d0102725f6d17c16b8cc1a581dcd0a2ac7c3a95ce2b174a71306a4884933
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from sentence_transformers import SentenceTransformer, models, util
import nltk
nltk.download('punkt')
import glob
import os
from tqdm.notebook import tqdm_notebook
import pickle

In [ ]:
l_sents= []
sents = set()
for filename in tqdm_notebook(glob.glob(f'/content/drive/My Drive/corpus/*.txt')):
    sents = set()
    with open(filename, 'r', encoding="utf8", errors='ignore') as in_file: 
        text = in_file.read().replace('\n', ' ')
    sents |= set(nltk.sent_tokenize(text))
    sents = list(sents)
    _, tail = os.path.split(filename)
    auth = tail.split("_")[0]
    l_sents.append((auth,sents,tail))

l_sents = list(l_sents)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
from os.path import exists

PATH = '/content/drive/My Drive/encodings/'
embeddings_l = []
authors_d = {}
authors_l = []
for i in tqdm_notebook(l_sents):
    filename = PATH + i[2] + '.pickle'
    if exists(filename): # load
        with open(filename, 'rb') as f:
            embeddings = pickle.load(f)
    else: # calculate and save
        embeddings = model.encode(i[1])
        with open(filename, 'wb') as f:
            pickle.dump(embeddings, f, pickle.HIGHEST_PROTOCOL)
    embeddings_l.append(embeddings)
    if not i[0] in authors_d:
        authors_d[i[0]] = len(authors_d)
    authors_l.append(authors_d[i[0]])


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
import numpy as np
from math import log
from sklearn.linear_model import LinearRegression

In [ ]:
def box_counting(arr):
    ks = np.max(arr)
    norm = np.linalg.norm(arr)
    arr = (arr/ks) >=0.5
    x, y = arr.shape
    scales = range(1,y)
    wykresx = []
    wykresy = []
    for k in scales:
        S = np.add.reduceat(np.add.reduceat(arr, np.arange(0, x, k), axis=0),np.arange(0, y, k), axis=1)
        wykresx.append( log(k) )
        wykresy.append( log(np.sum(S>0) ))
    wykresx = np.array(wykresx).reshape(-1,1)
    wykresy = np.array(wykresy)
    reg = LinearRegression().fit(wykresx, wykresy)
    return abs( reg.coef_[0] )


In [ ]:
fdimensions = []
for i in tqdm_notebook(embeddings_l):
    fdimensions.append(box_counting(i))

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
fdimensions

[1.3052034689695788,
 1.3683570921129347,
 1.3963334526877884,
 1.3888534176041194,
 1.4505258274244028,
 1.3524461911842158,
 0.8473369011548643,
 1.44007881408748,
 1.462531373365843,
 1.1662574071325587,
 1.3793397367692861,
 1.375464968373771,
 1.3528708747228213,
 1.3473965460873614,
 1.3487555384709258,
 1.4143755212452473,
 1.4032220156830266,
 1.330837536581587,
 1.4308339468454252,
 1.294455733772043,
 1.3949324699533825,
 1.4962071104113035,
 1.516652710938774,
 1.405095325605253,
 1.2715097492905196,
 1.2991382994504896,
 1.3809063043959993,
 1.4202867434483677,
 1.4129862944626173,
 1.3755564323545997,
 1.412544784955455,
 1.342775961036524,
 1.4218850338938045,
 1.389188447047523,
 1.3648562635785435,
 1.4135621786495742,
 1.3667205366450679,
 1.362173254197152,
 1.0939144245324146,
 1.433348200518853,
 1.4394100756526063,
 1.43285477395286,
 1.3968206134959313,
 1.0663564658184495,
 1.313196050891972,
 1.336533475426311,
 1.407418019574558,
 1.0123235054854014,
 1.4652930

**---------------------------------------------------Z order, not used rn ----------------------------------------------------**

In [ ]:
W = 384 #width
#H = 1000 #height normalised (to 0-255, ale u nas inaczej)
Km = 0
Ka = []
X = 0
Y = 0
dane = 0
A = []

Patrzymy na każdą komórkę jako wektor [X, Y, V] oznaczające współrzędne x i y i wartość komórki

In [ ]:
def a2bin(a,pad=0):
    binar = "{0:b}".format(a)
    binar = binar.rjust(pad, "0")
    return binar

In [ ]:
a2bin(20)

'10100'

In [ ]:
bin(21)[0]

'0'

In [ ]:
def clz(a):
    i=0
    while a[i] == '0':
        i=i+1
    return i+1

In [ ]:
def interleaving(a, b,c): #change into LOT or MagicBits!
    a, b, c = a2bin(a,386), a2bin(b,386), list(map(a2bin(386), bytearray(c, "utf-8")))
    res = ''
    for i in range(386):
        res += a[i]
        res += b[i]
        for j in range(384):
            res+= c[j][i]

    return res


Pseudokod wszystkiego:

In [ ]:
for i in doc:
    x1 = sth * (X/W)
    y1 = sth * (H/W)
    Km = interleaving(x1, y1, dane)
    Ka.append(Km)
Ka = sort(Ka)
for i in range(1, len(Ka)) :
    c = 1/NUMBER(Ka[i] ^ Ka[i-1]) # NUMBER to prawdopodobnie liczba bitów Km (w pracy NUMBER=8)
    for i in range(c, NUMBER):
        A[i] += 1